In [ ]:
import torch
import torch.cuda
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [ ]:
# Load and normalizde the data

transform = transforms.Compose(
    #[transforms.Resize(227),
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize = 5
validSize = 0.2 # use 20% of train set as validation

trainValidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainSet, validSet = torch.utils.data.random_split(trainValidSet, [int(len(trainValidSet)*(1-validSize)), int(len(trainValidSet)*validSize)])

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {'cuda' if torch.cuda.is_available() else 'cpu'}")

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Writer for tensorBoard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
def trainNetwork(network, optimizer, lossFunction, epochs, writer, device, name):
    network.train()
    for epoch in range(epochs):
        
        ### TRAINING ###
        trainLoss = 0
        correctTrain = 0
        totalTrain = 0
        for batch_nr, (images, labels) in enumerate(trainLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  

            # Predict
            predictions = network(images)

            # Get loss and backpropogate
            loss = lossFunction(predictions, labels)
            loss.backward() 

            # Optimize parameters (weights and biases) and remove gradients after
            optimizer.step() 
            optimizer.zero_grad()

            # Save loss for whole epoch
            trainLoss += loss.item()
            
            # Calculate training accuracy
            _, predictions = torch.max(predictions, 1) 
            correctTrain += (predictions == labels).sum().item() 
            totalTrain += len(images)

            #print(f'Epoch [{epoch+1}/{epochs}] Batch [{batch_nr}/{len(trainLoader)}]')
        

        trainLoss /= len(trainLoader)
        trainAccuracy = 100 * correctTrain / totalTrain
        writer.add_scalar(f"Loss/train - {name}", trainLoss, epoch)
        writer.add_scalar(f"Accuracy/train - {name}", trainAccuracy, epoch)

        ### VALIDATION ###
        validLoss = 0
        correctValid = 0
        totalValid = 0
        for batch_nr, (images, labels) in enumerate(validLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  

            # Predict
            predictions = network(images)

            # Get loss
            loss = lossFunction(predictions, labels)

            # Save loss for whole epoch
            validLoss += loss.item()

            # Calculate vaildation accuracy
            _, predictions = torch.max(predictions, 1) 
            correctValid += (predictions == labels).sum().item() 
            totalValid += len(images)

            #print(f'Epoch [{epoch+1}/{epochs}] Batch [{batch_nr}/{len(validLoader)}]')

        validLoss /= len(validLoader)
        validAccuracy = 100 * correctValid / totalValid
        writer.add_scalar(f"Loss/valid - {name}", validLoss, epoch)
        writer.add_scalar(f"Accuracy/valid - {name}", validAccuracy, epoch)

        # Print reuslt of epoch
        print(f'Epoch [{epoch+1}/{epochs}] \t Training Loss: {trainLoss} \t Validation Loss: {validLoss} \t Traning Acc: {trainAccuracy}% \t Validation Acc: {validAccuracy}%')

    writer.flush()

In [ ]:
def testNetwork(network, device):
    network.eval()
    correctTest = 0
    totalTest = 0

    ### TESTING ###
    with torch.no_grad(): 
        for batch_nr, (images, labels) in enumerate(testLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  
            
            # Get predictions and get the amount of correct predicitons
            predictions = network(images)
            _, predictions = torch.max(predictions, 1) 
            correctTest += (predictions == labels).sum().item() 
            totalTest += len(images)
            
            #print(f'Batch [{batch_nr}/{len(testLoader)}]', end='\r')
        
    print(f"Test Accuracy: {100 * correctTest / totalTest}%")

In [ ]:
class myAlexNet(nn.Module):
    def __init__(self, pretrained):
        super(myAlexNet,self).__init__()
        self.alexNet = torchvision.models.alexnet(pretrained=pretrained)
        self.out = nn.Linear(1000,10)
    
    def forward(self, x):
        x = F.relu(self.alexNet.forward(x))
        return self.out(x)


In [ ]:
learningRate = 0.005
epochs = 4

In [ ]:
networkNotTrained = myAlexNet(pretrained=False).to(device)

In [ ]:
optimizerAdam = torch.optim.Adam(networkNotTrained.parameters(), lr=learningRate)
lossFunction = nn.CrossEntropyLoss().to(device)

In [ ]:
networkNotTrained.eval()

In [ ]:
# trainNetwork(networkNotTrained, optimizerAdam, lossFunction, epochs, writer, device, "Not pretrained")

In [ ]:
networkPretrained = myAlexNet(pretrained=True).to(device)

In [ ]:
for param in networkPretrained.alexNet.parameters():
    param.requires_grad=False

In [ ]:
trainNetwork(networkPretrained, optimizerAdam, lossFunction, epochs, writer, device, "Pretrained weights")

In [ ]:
testNetwork(networkPretrained, device)

In [ ]:
# Load pre-trained AlexNet model
alexnet = torchvision.models.alexnet(pretrained=True)

# Replace last layer with new layer with 10 output classes
alexnet.classifier[6] = torch.nn.Linear(4096, 10)

# Freeze all layers except the last layer
for param in alexnet.parameters():
    param.requires_grad = False
for param in alexnet.classifier[6].parameters():
    param.requires_grad = True

alexnet.to(device)


In [ ]:
trainNetwork(alexnet, optimizerAdam, lossFunction, epochs, writer, device, "Pretrained weights")


Epoch [1/4] 	 Training Loss: 2.371517862394452 	 Validation Loss: 2.3709585531949995 	 Traning Acc: 10.8475% 	 Validation Acc: 10.86%
Epoch [2/4] 	 Training Loss: 2.3722538097053767 	 Validation Loss: 2.368909815609455 	 Traning Acc: 10.7175% 	 Validation Acc: 11.63%
Epoch [3/4] 	 Training Loss: 2.3727849128097294 	 Validation Loss: 2.374996145546436 	 Traning Acc: 10.8475% 	 Validation Acc: 11.04%
